## Non-Dominated Sorting Genetic Algorithm II (NSGA-II)

NSGA-II can be used to solve a multivariate, multi-objective optimization problems. For instance, here the objective is to minimize two objectives, a beam’s weight and deflection (functions f1 and f2), by finding a good combination of two variables, the beam's diameter, d, and length, l. 

When you run the code below, NSGA-II will look for a good solution for the two objectives. 

While setting up the optimisation algorithm, the user will have an idea about the possible range of each variable (d and l). For instance, here the diameter can be between 0.01 m and 0.05 m while the length can be between 0.2 m and 1.0 m. 

## Exercise 4

- 4.1. Run the code below and observe the results you get from this type of genetic algorithm. What is the benefit of obtaining the first Pareto front rather than a single optimal solution?
- 4.2. Try different values of ROW, P and E. How does this affect the first Pareto front? 
- 4.3. In the next block of code, "Constraint handling", try changing the the maximum stress and see how this changes the subset of the first Pareto front that the designer can choose from.

In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import math
import numpy as np

%run problem.ipynb
%run evolution.ipynb

ROW = 7800 # [kg/m^3] density of beam
P = 2000 # [N] force applied to the end of the beam 
E = 207e9 # [Pa] young's modulus of beam

def f1(d, l):
    # minimize weight
    return ROW * l * (np.pi * d**2) / 4.


def f2(d, l):
    # minimize deflecton
    return 64. * P * l**3 / (3. * E * np.pi * d**4)

# variable range:
## d between 0.01 m and 0.05 m
## l between 0.2 m and 1.0 m
problem = Problem(num_of_variables=2, objectives=[f1, f2], variables_range=[(0.01, 0.05), (0.2, 1.0)], same_range=False, expand=True)
evo = Evolution(problem, mutation_param=20, num_of_generations=100)
func = []
features = []
for i in evo.evolve():
    # print(i, i.objectives, i.features)
    func.append(i.objectives)
    features.append(i.features)

function1 = [i[0] for i in func]
function2 = [i[1] for i in func]
plt.xlabel('Function 1: weight [kg]', fontsize=15)
plt.ylabel('Function 2: deflection [m]', fontsize=15)
for i in range(len(features)):
    plt.plot(function1[i], function2[i], '.g')
plt.show()


Exception: File `'problem.ipynb'` not found.

## Pareto front

The green points in the plot above are the first Pareto front of the solutions found. This means that none of the green points are dominated by any of the other green points, but they all dominate every other solution that was found. The first Pareto front is therefore the set of solutions that are all as good as each other, i.e. there is not a single optimal point. This is because we are trying to minimise two objectives at the same time and, as you can see, reducing the deflection results in a higher weight and vice versa. 

## Constraint handling

But then how does the designer choose from this set of good solutions? The choice can be narrowed down by looking at the constraints such as maximum stress and maximum deflection of the beam. If the maximum stress is 300 MPa and the maximum deflection is 0.005 m then we can use these to eliminate some of the first Pareto front solutions. 

Run the code below:

In [ ]:
S_y = 300e6 # [Pa] # maximum stress
DELTA_MAX = 0.005 # [m] maximum deflection

def stress(d, l):
    return 32. * P * l / (np.pi * d**3)

function1 = [i[0] for i in func]
function2 = [i[1] for i in func]
plt.xlabel('Function 1: weight [kg]', fontsize=15)
plt.ylabel('Function 2: deflection [m]', fontsize=15)
plt.plot([0, 3],[DELTA_MAX, DELTA_MAX], '--r')
plt.text(1.75, DELTA_MAX*1.1, f'Max Deflection={DELTA_MAX}m', color='r')

# check max stress constraint. max stress <= S_y
for i in range(len(features)):
    if stress(features[i][0], features[i][1]) > S_y:
        plt.plot(function1[i], function2[i], '.r')
    else:
        plt.plot(function1[i], function2[i], '.g')
g_patch = mpatches.Patch(color='g', hatch=".", label='Stress(d,l) <= max')
r_patch = mpatches.Patch(color='r', hatch=".", label='Stress(d,l) > max')
plt.legend(handles=[g_patch, r_patch])
# plt.scatter(function1, function2)
plt.show()

In the above code, we first calculated the stress in the beam for each pair of variables in the first Pareto front. The stresses were compared with the maximum stress and coloured red if they are greater. The maximum deflection was also plotted as the red dashed line. All solutions that met the stress requirements also met the deflection requirements and are coloured green. 


# Zoomed in

Zooming in to the 2-objective Pareto front in the desirable stress region (green), we can see that there is still not a single optimal combination of the diameter and length of the beam. Although the deflection difference is small, there is still a trade-off between reducing the deflection and increasing the weight of the beam.

In [ ]:
plt.xlabel('Function 1: weight [kg]', fontsize=15)
plt.ylabel('Function 2: deflection [m]', fontsize=15)

# check max stress constraint. max stress <= S_y
for i in range(len(features)):
    if stress(features[i][0], features[i][1]) > S_y:
        plt.plot(function1[i], function2[i], '.r')
    else:
        plt.plot(function1[i], function2[i], '.g')
g_patch = mpatches.Patch(color='g', hatch=".", label='Stress(d,l) <= max')
r_patch = mpatches.Patch(color='r', hatch=".", label='Stress(d,l) > max')
plt.legend(handles=[g_patch, r_patch])
plt.ylim([0,0.002])
# plt.scatter(function1, function2)
plt.show()